<a href="https://colab.research.google.com/github/ACRainieri/pyspark/blob/main/Projeto_WEB_DSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Flask


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def index():
  return "Index!"

@app.route("/hello")
def hello():
  return "Hello World!!"

@app.route("/members")
def members():
  return "Members"

if __name__ == "__main__":
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
